# GPU

In [1]:
!nvidia-smi

Sat Feb  8 23:02:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.02              Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   43C    P8              1W /  125W |      31MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# 计算设备

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x7ff3024dd970>)

# 查询可用GPU的数量

In [3]:
torch.cuda.device_count()

1

# 这两个函数允许我们在请求的GPU不存在的情况下运行代码

In [4]:
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

# 查询张量所在的设备

In [6]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

# 存储在GPU上

In [7]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

# 第二个GPU上创建一个随机张量

In [8]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.4617, 0.6066, 0.6422],
        [0.0939, 0.3558, 0.7009]])

# 要计算X+Y，我们需要决定在哪里执行这个操作

In [10]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


# 数据在同一个GPU上，才可以相加

In [11]:
Y + Z

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [12]:
Z.cuda(0) is Z

True

# 神经网络与GPU

In [13]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)

tensor([[-0.4582],
        [-0.4582]], device='cuda:0', grad_fn=<AddmmBackward0>)

# 确认模型参数存储在同一个GPU上

In [14]:
net[0].weight.data.device

device(type='cuda', index=0)